In [1]:
import pandas as pd
import numpy as np
import config
import datetime
from sqlalchemy import text
import plotly
import plotly.express as px
import ipywidgets
from urllib.request import urlopen
import json
import plotly.graph_objects as go
import geopandas as gpd
import warnings
import pyproj
import plotly.io as pio
import pickle
print(config.version)
print(config.connection_string)
print(plotly.__version__, ipywidgets.__version__, pd.__version__, np.__version__)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', category=UserWarning) 

3.2.0.Dev
postgresql://postgres:sa@localhost/icip
6.3.1 8.1.7 2.0.0 1.26.4


In [2]:
# gdf = gpd.read_file("../CA/contours-iris.gpkg")
# gdf = gdf.to_crs(epsg=4326)
# gdf["fid"] = gdf.index
# gdf["lon"] = gdf.geometry.centroid.x
# gdf["lat"] = gdf.geometry.centroid.y
# gdf = gdf.sort_values(by="code_iris")
# with open("gdf.pickle", "wb") as f:
#     pickle.dump(gdf, f)
with open("gdf.pickle", "rb") as f:
    gdf_france = pickle.load(f)
    gdf_france['geometry'] = gdf_france["geometry"].simplify(tolerance=0.001)
    geojson=gdf_france.__geo_interface__


In [3]:
spe_df = pd.read_sql("select id, label_long label from specialite", config.connection_string, index_col="id")
spe_df.T

id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
label,Psychiatre,Anesthésiste,Dermatologue,Gastro-entérologue,Gynécologue,Opthtalmogue,Pédiatre,Radiologue,Dentiste,Généraliste,Cardiologue,Chirugien,Endocrinologue,Gériatre,Neurologue,Oto-rhino-laryngologiste,Pneumologue,Radiothérapeuthe,Rhumatologue,Stomatologue


In [12]:
specialite = 10
year = 20 # 20 to 25
source="PA" # PA or PS
time=30 # 30 or 45
time_type="HC" # HC or HP


In [13]:
sql = f"""
select * from apl.apl_study s
where s.specialite_id={specialite}
and s.year={year}
and s.source='{source}'
and s.time={time}
and s.time_type='{time_type}'
order by date desc
limit 1
"""
study_df = pd.read_sql(sql, config.connection_string)
study_df

,year,specialite_id,source,time,time_type,exp,mean,std,q10,q25,q50,q75,q90,min,max,date,key
0,20,10,PA,30,HC,-0.12,69.297407,37.799369,31.187375,42.506375,59.518193,88.822859,123.141263,0.0,729.717071,2025-10-29 22:05:59.576749,587733453917


In [14]:
key = study_df["key"].iloc[0]
print(f"Study key: {key}")
sql=f"""
select * from apl.apl a
where a.study_key = {key}
"""
apl = pd.read_sql(sql, config.connection_string)
len(apl)

Study key: 587733453917


48569

In [15]:
prop = (apl["pop_gp"]/apl["pop"]).replace(np.nan, 0)
np.mean(prop)

1.0032632827114933

In [16]:
gdf=gdf_france
apl.loc[apl["iris_string"]=="180331201","apl"]=apl[apl["iris_string"]=="181740000"]["apl"].iloc[0]
apl.loc[apl["iris_string"]=="140110000","iris_string"]="145810000"

In [17]:
apl["code_iris"]=apl["iris_string"]
gdf = gdf.merge(apl, on="code_iris", how="left", suffixes=('', '_dest')).copy()
gdf["pretty"]=gdf["apl"].fillna(0).apply(lambda x: round(x, 0)).astype(np.int32)
qmax = study_df["max"].iloc[0] if study_df["max"].iloc[0] != 0 else 1.0
q90 = study_df["q90"].iloc[0] if study_df["q90"].iloc[0] != 0 else 0.0
q75 = study_df["q75"].iloc[0] if study_df["q75"].iloc[0] != 0 else 0.0
q50 = study_df["q50"].iloc[0] if study_df["q50"].iloc[0] != 0 else q75
q25 = study_df["q25"].iloc[0] if study_df["q25"].iloc[0] != 0 else q50
q10 = study_df["q10"].iloc[0] if study_df["q10"].iloc[0] != 0 else q25

len(gdf)

48569

In [18]:
fig = go.Figure()
fig.add_trace(go.Choropleth(
    geojson=geojson,
    locations=gdf["fid"],
    z=gdf["apl"],
    zmin=0,
    zmax=qmax,
    hoverinfo="skip",
    showscale=False,
    # colorscale="Portland",
    colorbar={"title": "APL",
              # "tickvals": [0, np.log10(5), 1, np.log10(50), 2, np.log10(200), 3],
              # "ticktext": ['0', '5', '10', '50', '100', '200', '1000'],
             },
    colorscale=[[0.0, "rgb(64,64,127)"],
                [q10/qmax, "rgb(112,112,127)"],
                [q25/qmax, "rgb(159,159,127)"],
                [q50/qmax, "rgb(255,255,127)"],
                [q75/qmax, "rgb(209,127,79)"],
                [q90/qmax, "rgb(187,64,55)"],
                [0.99, "rgb(165,0,32)"],
                [1.0, "rgb(127,0,0)"]],
    marker = dict(line=dict(width=0))
))
center_lat = gdf.geometry.centroid.y.mean()
center_lon = gdf.geometry.centroid.x.mean()
fig.update_layout(
    geo=dict(
        projection_type="mercator",
        center={"lat": center_lat, "lon": center_lon},
        fitbounds="locations",
        showland=False,
        showcountries=False,
        showocean=False,
        showlakes=False,
        showrivers=False,
        visible=False
    ),
    title=f"APL France",
    height=1000,
    width=1200,
    margin=dict(l=0, r=0, t=50, b=0),
)
"End"
# fig.show()

'End'

In [19]:
pio.write_html(fig, file=f"apl_france_{specialite}_{year}.html", auto_open=True, include_plotlyjs="cdn", full_html=True, config={"responsive": True})